In [1]:
import os
os.chdir('../..')

In [2]:
%load_ext autoreload
%autoreload 2
from mars.db import collections
from mars.db import db
from mars import search
import pandas as pd
import ast
from tqdm import tqdm

In [3]:
search.search_for?

In [4]:
df_papers = pd.read_csv('data/s2orc/ai_papers_meta.csv', index_col=0)

/home/sgizinski/repos/AI-strategies-papers-regulations-monitoring/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df_search_results = pd.read_csv("results.csv", index_col=0)

In [6]:
papers_matched = set(df_search_results.index.unique())

In [7]:
df_papers_filtered = df_papers[df_papers['paper_id'].isin(papers_matched)].copy()

In [8]:
# get last name of first author
names = df_papers_filtered['authors'].apply(ast.literal_eval).str[0].apply(pd.Series)['last']
df_papers_filtered['author_name']=names

In [9]:
found_authors=dict()
for index, row in tqdm(df_papers_filtered.iterrows(), total=len(df_papers_filtered)):
    try:
        i = row['paper_id']
        author=row['author_name']
        found_authors[i] = search.search_for(author)
    except AttributeError:
        found_authors[i]=[]

100%|█████████████████████████████████████████████████████████████████████████████████| 206/206 [00:28<00:00,  7.16it/s]


In [12]:
df_result = pd.DataFrame(pd.Series(found_authors)).explode(0)[0].apply(pd.Series)
del df_result['embedding']
del df_result['_rev']

In [16]:
df_result = df_result.reset_index()

In [26]:
df_authors = df_result[['index','source_doc_id']]

In [29]:
df_authors['authors_match'] = df_result['sentence']
df_

0                        Bickel P, Arampatzis G, Burges...
1                        Cameron, I., Synnott, J., Beis...
2        [80] J. H. Metzen, M. C. Kumar, T. Brox, and V...
3                        Litsky D., Kumar S. and Lespri...
4        This could not have been possible without the ...
                               ...                        
19211    Prival, M.J., Bell, S.J., Mitchell, V.D., Reip...
19212     ● Ronald C. Arkin (Founding Co-Chair) – Regen...
19213    http://www.aisb.org.uk/public-engagement/what-...
19214    Gaussian A probability density which adopts a ...
19215    Sir John Bell, Regius Professor of Medicine at...
Name: sentence, Length: 19216, dtype: object

In [27]:
df_authors

,index,source_doc_id
0,37359148,Documents/351905
1,37359148,Documents/273922
2,5737015,Documents/262636
3,199006245,Documents/354879
4,199006245,Documents/31262414
...,...,...
19211,65218685,Documents/346003
19212,65218685,Documents/32568525
19213,65218685,Documents/32568084
19214,65218685,Documents/32568515


In [20]:
df_search_results.reset_index() = df_search_results
df_titles = df_search_results.reset_index()[['index','source_doc_id']]

In [30]:
both = pd.merge(df_authors,df_titles, on=['index', 'source_doc_id'])

In [32]:
a = both.drop_duplicates()

In [35]:
a.source_doc_id.drop_duplicates()

0      Documents/262636
1      Documents/259873
7      Documents/265537
18     Documents/265783
57     Documents/262470
58     Documents/258530
60     Documents/262177
71     Documents/257444
127       Documents/926
128    Documents/262754
Name: source_doc_id, dtype: object